In [ ]:


# This script is to merge DEM rasters and clip useful domain
work_dir='/glade/u/home/hongli/work/research/discretization/scripts'
input_dir=${work_dir}/input
slp_file=${input_dir}/slope.tif
asp_file=${input_dir}/aspect.tif

# reclassify slope
# Define bins that you want, and then classify the data
class_bins = [lidar_chm_im.min(), 2, 7, 12, np.iinfo(np.int32).max]

# Classify the original image array, then unravel it again for plotting
lidar_chm_im_class = np.digitize(lidar_chm_im, class_bins)

# Note that you have an extra class in the data (0)
print(np.unique(lidar_chm_im_class))
[0 1 2 3 4]